# Grouping items on warehouseid and inserting it into mongodb

In [1]:
from pymongo import MongoClient 
import pandas as pd
import sys
import numpy as np

In [2]:
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate

In [3]:
try: 
    conn = MongoClient('192.168.1.72', 27017) 
    print("Connected successfully!!!") 
except:   
    print("Could not connect to MongoDB") 


Connected successfully!!!


### Reading excel file

In [4]:
dfIM = pd.read_excel('ItemMastersData1.xlsx')

In [5]:
dfIM

,itemname,ItemId,Number,WarehouseId,WarehouseName,active
0,Kisan Kirana Kabuli Chana 5 Kg,23,AE10123,1,MP-INDB-1,1
1,Ghari Soap 5 MRP,138,AT104103,1,MP-INDB-1,1
2,Ghari Soap 5 MRP,140,AT104103,1,MP-INDB-1,1
3,Ghari Detergent 10 MRP 150 gm,144,AT104109,1,MP-INDB-1,1
4,Ghari Detergent 10 MRP 150 gm,145,AT104109,1,MP-INDB-1,1
...,...,...,...,...,...,...
8884,Santoor Super Saver White 40 MRP 4*60 Combo,350515,BA2051359,9,MP-BPL-1,1
8885,Santoor Super Saver White 94 MRP 4*100 Combo,350682,BA2051360,9,MP-BPL-1,1
8886,Parle Krack Jack 10 MRP,351899,AW166296,55,GJ-AMD-1,1
8887,Parle Krack Jack 10 MRP,351915,AW166296,78,GJ-AMD-2,1


In [6]:
#dfIM.drop(['B', 'C'], axis=1)

In [7]:
cIM = dfIM.groupby('WarehouseId')['ItemId'].apply(set)
cIM = cIM.to_frame()
cIM.reset_index()

,WarehouseId,ItemId
0,1,"{23, 217126, 178227, 13443, 39006, 266342, 178..."
1,6,"{3098, 3099, 3101}"
2,7,"{194583, 217127, 4138, 4139, 4140, 4143, 4146,..."
3,9,"{178176, 38931, 8214, 8220, 8221, 264185, 2744..."
4,10,"{10240, 10241, 10242, 178177, 268290, 10245, 1..."
5,11,"{12289, 178178, 12291, 268291, 12295, 12297, 2..."
6,12,"{178179, 14364, 14365, 14367, 14368, 14369, 14..."
7,33,"{178197, 77855, 77861, 77862, 77863, 96303, 77..."
8,52,"{331837, 217168, 176217, 225374, 133244, 13324..."
9,53,"{194569, 141328, 217105, 139288, 178216, 14137..."


In [8]:
cIM['ItemId'] = [list(i) for i in cIM['ItemId']]

In [9]:
cIM.rename(columns={"WarehouseId": "WarehouseId", "ItemId": "item"})

,item
WarehouseId,
1,"[23, 217126, 178227, 13443, 39006, 266342, 178..."
6,"[3098, 3099, 3101]"
7,"[194583, 217127, 4138, 4139, 4140, 4143, 4146,..."
9,"[178176, 38931, 8214, 8220, 8221, 264185, 2744..."
10,"[10240, 10241, 10242, 178177, 268290, 10245, 1..."
11,"[12289, 178178, 12291, 268291, 12295, 12297, 2..."
12,"[178179, 14364, 14365, 14367, 14368, 14369, 14..."
33,"[178197, 77855, 77861, 77862, 77863, 96303, 77..."
52,"[331837, 217168, 176217, 225374, 133244, 13324..."


In [10]:
cIM2 = cIM.reset_index()

In [11]:
%time data_dict = cIM2.to_dict('records')

Wall time: 0 ns


#### reading the data from database


In [4]:
mydb = conn.SK
collection1 = mydb.OrderMaster
%time df1 = pd.DataFrame(list(collection1.find({},{"Skcode":1,"orderDetails":1,"WarehouseId":1,"_id":0})))

Wall time: 38.7 s


In [5]:
df1

,Skcode,WarehouseId,orderDetails
0,SK8393,1,[{'GUID': f7667b25-5aea-0e48-8c99-f7105851cd50...
1,SK10454,7,[{'GUID': d6f02625-f65d-eb47-807b-5f6a37984c4f...
2,SK4160,1,[{'GUID': 5d89d8c5-985d-0441-80ec-f4569d641d8c...
3,SK9511,1,[{'GUID': 8257ebaa-8d5f-d74c-9bae-9d4842d08cbe...
4,SK8599,1,[{'GUID': 50bf8fda-7076-a849-96b6-034e024deb4c...
...,...,...,...
234779,SK10612,9,[{'GUID': 65804773-fbab-c546-89e8-1ff425b83fcc...
234780,SK32014,55,[{'GUID': 3629b028-d1f6-6d41-9c23-c277d2325ecd...
234781,SK64077,33,[{'GUID': d997747c-9aa6-b54d-8973-c972413266e6...
234782,SK15021,10,[{'GUID': 720eac9e-f973-bd4e-b122-89fc9a3195df...


#### extracting itemid's from orderDetails and exploding it using explode function

In [6]:
df1["item"] = [[i[j]['ItemId'] for j in range(len(i))] for i in df1['orderDetails']]
df1 = df1[['Skcode','item',"WarehouseId"]]
df2 = df1.explode('item')

In [15]:
df2

,Skcode,item,WarehouseId
0,SK8393,1576,1
0,SK8393,923,1
0,SK8393,933,1
0,SK8393,1197,1
0,SK8393,1784,1
...,...,...,...
234781,SK64077,77801,33
234781,SK64077,77306,33
234781,SK64077,77067,33
234782,SK15021,31937,10


In [16]:
df3 = df2.dropna()

In [19]:
test=df3.loc[df3.WarehouseId == 7]

In [22]:
test['item'].min()

51

In [9]:
df3.item.nunique()

14669

In [10]:
a=df3.groupby('WarehouseId')['item'].apply(set)
b = a.to_frame()
c = b.copy()
c1 = c.reset_index()
c1

,WarehouseId,item
0,0,"{23812, 38789, 39049, 23662, 39543, 23672, 391..."
1,1,"{0, 1, 2, 6, 23, 48, 49, 50, 51, 52, 53, 54, 5..."
2,7,"{51, 8249, 8250, 60, 8254, 8258, 8262, 8266, 8..."
3,8,"{5656, 5653}"
4,9,"{8204, 8205, 8206, 8208, 8209, 8214, 8215, 821..."
5,10,"{10240, 10241, 10242, 10243, 268290, 10245, 10..."
6,11,"{12289, 12290, 10242, 12291, 147458, 268291, 1..."
7,12,"{219147, 8208, 14358, 14359, 38934, 14363, 143..."
8,13,"{38337, 18338, 38324}"
9,14,"{19087, 18580, 20847}"


In [39]:
c1['item'] = [list(i) for i in c1['item']]

## insert into warehouse 

In [27]:
%time data_dict = c1.to_dict('records')

Wall time: 1 ms


In [12]:
try: 
    conn = MongoClient('192.168.1.101', 27017) 
    print("Connected successfully!!!") 
except:   
    print("Could not connect to MongoDB") 

Connected successfully!!!


In [13]:
mydb = conn.ankit_database
collection = mydb.ItemWarehouse

In [14]:
collection.remove() 


C:\Users\Ankit\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  """Entry point for launching an IPython kernel.


{'n': 169, 'ok': 1.0}

In [15]:
collection.insert_many(data_dict)